# Estimating Covariance Matrix of Force Distribution
We're trying to compute the probability that the uncertainty in any forces is above a certain threshold.
What we have is the distribution of forces predicted for each atom in each frame from several different models.

The first assumption is that the variance in forces between models is related to the expected error between the model and the process it is predicting.
Variances between models beyond a certain threshold indicate a high chance for error

The complicating element is that each component of each force is cross-correlated with the components of force. 
Some forces are strongly related, like those of atoms with the similar bonding environments,
and others are not.
We must address this cross-correlation by creating a multi-variate distribution of forces.

Our target is to make a multi-variate normal distribution.
For that, we need a covariance matrix between each component of force between each atom in each frame

In [1]:
from scipy.stats import multivariate_normal
import numpy as np

Config

In [2]:
num_frames = 7  # Number of frames in the trajectory
num_models = 5  # Number of models in our ensemble
num_atoms = 2  # Number of atoms in our simulation cell

Let's start with an example random matrix

In [3]:
force_preds = np.random.normal(size=(num_models, num_frames, num_atoms, 3))

Before we go further, let's reduce the dimensionality. We don't actually care if atoms are from the same frame or not, so let's collapse the matrix from 
(models, frames, atoms, 3) to (models, frames * atoms, 3)

In [4]:
force_preds = np.reshape(force_preds, (num_models, num_frames * num_atoms, 3))
force_preds.shape

(5, 14, 3)

The next thing we need is a covariance between each of these. [Numpy's covariance function ](https://numpy.org/doc/stable/reference/generated/numpy.cov.html) only works with 2D matrices, where rows are different variables (here forces in specific directions) and columns are observations (different models)

In [5]:
force_preds_flat = np.reshape(force_preds, (num_models, num_frames * num_atoms * 3))
force_preds_flat.shape

(5, 42)

In [6]:
force_cov = np.cov(force_preds_flat.T)
force_cov.shape

(42, 42)

> Note: This matrix is arranged where the indices go (frame_0_atom_0_force_x, frame_0_atom_0_force_y, ..., frame_0_atom_1_force_x, ..., frame_1_atom_0_force_x)

Now that we have the covariance matrix, we can make the normal distribution

In [7]:
force_err_dist = multivariate_normal(cov=force_cov, allow_singular=True)

This distribution allows us to sample an expected variance for all components of force for all atoms, 
which represent (by our assumption that ensemeble variance is related to model error) samples from a distribution of expected errors between the model.

In [8]:
force_var_samples_flat = force_err_dist.rvs(100)
force_var_samples_flat.shape

(100, 42)

We can determine the magnitude of expected error for the force on each atom by restoring the (n_atoms, 3) structure from the original data

In [9]:
force_var_samples = np.reshape(force_var_samples_flat, (-1, num_atoms * num_frames, 3))
force_var_samples.shape

(100, 14, 3)

In [10]:
force_var_samples_mag = np.linalg.norm(force_var_samples, axis=-1)
force_var_samples_mag.shape

(100, 14)

Then we can do tests

In [11]:
any_above_thr = (force_var_samples > 1).any(axis=1).mean()
print(f'Probability that any atom is above the threshold: {any_above_thr}')

Probability that any atom is above the threshold: 0.8033333333333333
